In [1]:
import os
import pandas as pd
from astroquery.ipac.ned import Ned
from astropy.io import fits
from astropy.coordinates import SkyCoord
import astropy.units as u

In [21]:
# List of object names for redshift retrieval
object_names = [
    "1H0323+342", "1H1934-063", "3C120", "3C382", "3C390.3", "ARK120", "ARK564",
    "MCG-6-30-15", "MRK1018", "MRK110", "MRK279", "MRK335", "MRK509", "MRK590",
    "MRK79", "MRK841", "NGC1365", "NGC3783", "NGC4051", "NGC4151", "NGC4748",
    "NGC7469", "PG0804+761", "PG0844+349", "PG1426+015", "RBS1124", "UGC6728"
]

# Retrieve redshifts from NED
redshifts = {}
for object_name in object_names:
    try:
        result_table = Ned.query_object(object_name)
        redshift = result_table['Redshift'][0]
        redshifts[object_name] = redshift
    except Exception as e:
        redshifts[object_name] = 'N/A'
        print(f"Could not retrieve redshift for {object_name}: {e}")

# Function to parse RA and Dec from 'source' file
def parse_ra_dec(source_file):
    with open(source_file, 'r') as file:
        for line in file:
            if line.startswith('circle'):
                parts = line.split('(')[1].split(')')[0].split(',')
                ra = parts[0]
                dec = parts[1]
                return ra, dec
    return 'N/A', 'N/A'

# Function to convert RA and Dec to degrees
def convert_to_degrees(ra, dec):
    try:
        coord = SkyCoord(ra, dec, unit=(u.hourangle, u.deg))
        return coord.ra.degree, coord.dec.degree
    except Exception as e:
        print(f"Could not convert RA/Dec to degrees for {ra}, {dec}: {e}")
        return 'N/A', 'N/A'

# Function to extract required information from a FITS file
def extract_fits_info(file_path, source_file):
    with fits.open(file_path) as hdul:
        header = hdul[0].header
        header_1 = hdul[1].header
        
        # Extracting the required information
        object_name = header.get('OBJECT', 'N/A')
        target_name = header.get('TARG_ID', 'N/A')
        obs_id = header.get('OBS_ID', 'N/A')
        filter_name = header.get('FILTER', 'N/A')
        exposure = header_1.get('EXPOSURE', 'N/A')
        date_obs = header.get('DATE-OBS', 'N/A')
        
        # Get redshift from the pre-fetched redshifts dictionary
        redshift = redshifts.get(object_name, 'N/A')
        
        # Get RA and Dec from the source file
        ra, dec = parse_ra_dec(source_file)
        
        # Convert RA and Dec to degrees
        ra_deg, dec_deg = convert_to_degrees(ra, dec)
        
        return {
            'Name': object_name,
            'Target ID': target_name,
            'Observation ID': obs_id,
            'Filter': filter_name,
            'Exposure Time': exposure,
            'Date/Time of Observation': date_obs,
            'RA (J2000)': ra,
            'Dec (J2000)': dec,
            'RA (degrees)': ra_deg,
            'Dec (degrees)': dec_deg,
            'Redshift': redshift
        }

# Base directory containing the subdirectories
base_directory = '/mnt/home/guptaagr/AGN-Photometry/swift27/'
directories = []

# Collect relevant subdirectories
for file in os.listdir(base_directory):
    filename = os.fsdecode(file)
    if filename[0] != '0':
        continue
    else:
        directories.append(filename)
directories = sorted(directories)

# List to hold the extracted information
data = []

# Loop through each subdirectory and process the FITS files
for directory in directories:
    fits_dir = os.path.join(base_directory, directory, 'uvot', 'image')
    source_file = os.path.join(fits_dir, 'source')
    if os.path.exists(fits_dir) and os.path.exists(source_file):
        for filename in os.listdir(fits_dir):
            if filename.endswith('sk.img.gz'):
                file_path = os.path.join(fits_dir, filename)
                info = extract_fits_info(file_path, source_file)
                data.append(info)

# Create a DataFrame from the extracted data
df = pd.DataFrame(data)

# Uncomment the line below to save the DataFrame to a CSV file
df.to_csv('fits_info_with_z_ra.csv', index=False) 

print("Extraction complete. Data saved to 'fits_info_with_z.csv'.")

Extraction complete. Data saved to 'fits_info_with_z.csv'.


In [20]:
# List of object names for redshift retrieval
object_names = [
    "1H0323+342", "1H1934-063", "3C120", "3C382", "3C390.3", "ARK120", "ARK564",
    "MCG-6-30-15", "MRK1018", "MRK110", "MRK279", "MRK335", "MRK509", "MRK590",
    "MRK79", "MRK841", "NGC1365", "NGC3783", "NGC4051", "NGC4151", "NGC4748",
    "NGC7469", "PG0804+761", "PG0844+349", "PG1426+015", "RBS1124", "UGC6728"
]

# Retrieve redshifts from NED
redshifts = {}
for object_name in object_names:
    try:
        result_table = Ned.query_object(object_name)
        redshift = result_table['Redshift'][0]
        redshifts[object_name] = redshift
    except Exception as e:
        redshifts[object_name] = 'N/A'
        print(f"Could not retrieve redshift for {object_name}: {e}")

# Function to parse RA and Dec from 'source' file
def parse_ra_dec(source_file):
    with open(source_file, 'r') as file:
        for line in file:
            if line.startswith('circle'):
                parts = line.split('(')[1].split(')')[0].split(',')
                ra = parts[0]
                dec = parts[1]
                return ra, dec
    return 'N/A', 'N/A'

# Function to convert RA and Dec to degrees
def convert_to_degrees(ra, dec):
    try:
        coord = SkyCoord(ra, dec, unit=(u.hourangle, u.deg))
        return coord.ra.degree, coord.dec.degree
    except Exception as e:
        print(f"Could not convert RA/Dec to degrees for {ra}, {dec}: {e}")
        return 'N/A', 'N/A'

# Function to extract required information from a FITS file
def extract_fits_info(file_path, source_file):
    with fits.open(file_path) as hdul:
        header = hdul[1].header
        
        # Extracting the required information
        exposure = header.get('EXPOSURE', 'N/A')
        
#         # Calculate exposure time
#         if tstart != 'N/A' and tstop != 'N/A':
#             exposure_time = tstop - tstart
#         else:
#             exposure_time = 'N/A'
        
        # Get redshift from the pre-fetched redshifts dictionary
        redshift = redshifts.get(object_name, 'N/A')
        
        # Get RA and Dec from the source file
        ra, dec = parse_ra_dec(source_file)
        
        # Convert RA and Dec to degrees
        ra_deg, dec_deg = convert_to_degrees(ra, dec)
        
        return {
            'Exposure': exposure
        }

# Base directory containing the subdirectories
base_directory = '/mnt/home/guptaagr/AGN-Photometry/swift27/'
directories = []

# Collect relevant subdirectories
for file in os.listdir(base_directory):
    filename = os.fsdecode(file)
    if filename[0] != '0':
        continue
    else:
        directories.append(filename)
directories = sorted(directories)

# List to hold the extracted information
data = []

# Loop through each subdirectory and process the FITS files
for directory in directories:
    fits_dir = os.path.join(base_directory, directory, 'uvot', 'image')
    source_file = os.path.join(fits_dir, 'source')
    if os.path.exists(fits_dir) and os.path.exists(source_file):
        for filename in os.listdir(fits_dir):
            if filename.endswith('sk.img.gz'):
                file_path = os.path.join(fits_dir, filename)
                info = extract_fits_info(file_path, source_file)
                data.append(info)

# Create a DataFrame from the extracted data
df = pd.DataFrame(data)

# Uncomment the line below to save the DataFrame to a CSV file
df.to_csv('exposure.csv', index=False) 

print("Extraction complete. Data saved to 'fits_info_with_z.csv'.")

Extraction complete. Data saved to 'fits_info_with_z.csv'.


In [17]:
with fits.open('00096108001/uvot/image/sw00096108001uuu_sk.img.gz') as hdul:
    header = hdul[1].header

In [19]:
print(header)

XTENSION= 'IMAGE   '           / IMAGE extension                                BITPIX  =                  -32 / number of bits per data pixel                  NAXIS   =                    2 / number of data axes                            NAXIS1  =                 1104 / length of data axis 1                          NAXIS2  =                 1103 / length of data axis 2                          PCOUNT  =                    0 / number of random group parameters              GCOUNT  =                    1 / number of random groups                        EXTNAME = 'uu640832728I'       / Exposure ID                                    EXPID   =            640832728 / Exposure ID                                    BUNIT   = 'count   '           / physical units of the array values             HDUCLASS= 'OGIP    '           / Format conforms to OGIP standards              HDUCLAS1= 'IMAGE   '           / Image data                                     HDUCLAS2= 'TOTAL   '           / Total d

In [10]:
with fits.open('00096108001/uvot/image/sw00096108001uuu_sk.img.gz') as hdul:
    # Loop through all the headers in each extension
    for i, hdu in enumerate(hdul):
        print(f"Header for extension {i}:")
        print(hdu.header)

Header for extension 0:
SIMPLE  =                    T / file does conform to FITS standard             BITPIX  =                    8 / number of bits per data pixel                  NAXIS   =                    0 / number of data axes                            EXTEND  =                    T / FITS dataset may contain extensions            COMMENT   FITS (Flexible Image Transport System) format is defined in 'AstronomyCOMMENT   and Astrophysics', volume 376, page 359; bibcode: 2001A&A...376..359H TELESCOP= 'SWIFT   '           / Telescope (mission) name                       INSTRUME= 'UVOTA   '           / Instrument name                                DETNAM  = '1400    '                                                            FILTER  = 'U       '           / Filter                                         OBS_ID  = '00096108001'        / Observation ID                                 TARG_ID =                96108 / Target ID                                      SEG_NUM =       

In [15]:
with fits.open('00096108001/uvot/image/sw00096108001uuu_sk.img.gz') as hdul:
    header = hdul[1].header  # Accessing the first extension's header
    telapse = header.get('TELAPSE', 'Not found')
    exposure = header.get('EXPOSURE', 'Not found')
    
    print(f"TELAPSE: {telapse}")
    print(f"EXPOSURE: {exposure}")

TELAPSE: 54.7674199342728
EXPOSURE: 53.9036274841211
